# Value Iteration RL

- Start with a state-value function $v_0(s)$
- Iteratively:
    - Find the optimal state-value function by iteratively using the Bellman's optimality equation:
    
$$ v_{k+1}(s) \leftarrow \max_{a \in \mathcal{A}} \left( \mathcal{R}^a_s + \gamma \sum_{s^{\prime} \in \mathcal{S}} \mathcal{P}_{ss^{\prime}}^{a} v_k(s^{\prime}) \right) $$

      Or, in vector notation:
      
$$ \mathbf{v}_{k+1} \leftarrow \max_{a \in \mathcal{A}} \left( \mathcal{R}^a + \gamma \mathcal{P}^{a} \mathbf{v}_k \right), \; \mathcal{R}^a = \left[\mathcal{R}^a_s\right] \in \mathbb{R}^{\left| \mathcal{S} \right|}, \; \mathcal{P}^a = \left[\mathcal{P}^a_{ss^{\prime}}\right] \in \mathbb{R}^{\left| \mathcal{S} \right| \times \left| \mathcal{S} \right|} $$

      We should only stop when:
      
$$\left| v_{k+1}(s) - v_k(s) \right| < \epsilon, \; \forall s \in \mathcal{S}$$

- Finally, after we have discovered the optimal state-value function, we can derive from it the optimal action-value function, from which we can derive the optimal policy:

$$ \pi_{\star}(s) = \mathop{\mathrm{argmax}}_{a \in \mathcal{A}} q_{\star}(s, a) $$

$$ q_{\star}(s, a) = \mathcal{R}^a_s + \gamma \sum_{s^{\prime} \in \mathcal{S}} \mathcal{P}^{a}_{ss^{\prime}} v_{\star}(s^{\prime}) $$

    Or, once again in vector notation:
    
$$ \pi_{\star} = \mathop{\mathrm{argmax}}_{a \in \mathcal{A}} \left( \mathcal{R}^a + \gamma \mathcal{P}^a \mathbf{v}_{\star} \right) $$

# Grid-World Problem

Let's define a simple problem so that we can apply the above algorithm. This is a simple grid-world, where we want to reach any of the corner squares (the goal). We are allowed to move either up, down, left or right, and every state has a  a reward of $-1$, except of the goal states, regardless of the action taken.

Translating this problem into our **Markov Decision Process** $\mathcal{M} = \langle \mathcal{S}, \mathcal{A}, \mathcal{P}, \mathcal{R}, \gamma\rangle$:
- $\mathcal{S}$ is the set of all squares
- $\mathcal{A} = \{\mathtt{up}, \mathtt{down}, \mathtt{left}, \mathtt{right}\}$ if we are not in a goal state, otherwise $\mathcal{A} = \{\}$
- $\mathcal{P}$ is either 1 or 0 given the action and the two states, as this world is fully deterministic
- $\mathcal{R}^a_s = -1, \forall a \in \mathcal{A}$ if $s$ is not a goal state, otherwise $\mathcal{R}^a_s = 0$
- The discount factor $\gamma \in \left[0,1\right]$

In [1]:
class GridWorldMDP:
    
    def __init__(self, rows, columns, discount):   
        # Auxiliary variables
        self.rows = rows
        self.columns = columns
        
        # MDP Tuple
        self.states = [(i,j) for i in range(rows) for j in range(columns)]
        self.actions = ["up", "down", "left", "right"]
        self.probabilities = self.generate_probabilities(self.states, self.actions)
        self.rewards = self.generate_rewards(self.states)
        self.discount = discount
        
    def is_goal_state(self, state):
        # Get row and column from state
        i, j = state
        
        # Each of the possible goal states
        upper_left = (i == 0) and (j == 0)
        upper_right = (i == 0) and (j == self.columns-1)
        lower_left = (i == self.rows - 1) and (j == 0)
        lower_right = (i == self.rows - 1) and (j == self.columns - 1)
        
        return upper_left or upper_right or lower_left or lower_right
    
    def get_next_state(self, state, action):
        i, j = state
        
        # Change state according to action
        if action == "up":
            i_, j_ = max(0, i-1), j
        elif action == "down":
            i_, j_ = min(self.rows-1, i+1), j
        elif action == "left":
            i_, j_ = i, max(0, j-1)
        elif action == "right":
            i_, j_ = i, min(self.columns-1, j+1)
            
        return (i_, j_)
    
    def get_sucessor_states(self, state):
        if self.is_goal_state(state):
            s = []
        else:
            s = [self.get_next_state(state, action) for action in self.actions]
        return s
    
    def generate_probabilities(self, states, actions):
        # Populate only the possible transitions and its probabilities
        p = {}
        
        # For each state, populate only the adjacent states with probability 1
        for state in states:
            if not self.is_goal_state(state):
                for action in actions:
                    state_ = self.get_next_state(state, action)
                    p[(action, state, state_)] = 1
                
        return p
        
    def generate_rewards(self, states):
        r = {}
        
        # Populate the probabilities dictionary
        for state in states:
            if self.is_goal_state(state): r[state] = 0
            else: r[state] = -1
                
        return r

In [2]:
# Define world hyperparameters
rows = 10
columns = 10
discount = 0.9

# Generate world
MDP = GridWorldMDP(rows, columns, discount)